# torch를 이용한 angle 추정 모델 - dense_1st_torch
# 위에꺼 만들 때 사용함

In [15]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error

from torch.utils.tensorboard import SummaryWriter
import os
from os.path import join
from pickle import load
from tqdm import tqdm
import random
import datetime

In [16]:
######### 설정 영역 ########
modelVersion = 'Dense_1st_torch'
nameDataset = 'IWALQQ_2nd'
dataType = 'angle' # or moBWHT
# 데이터 위치
relativeDir = '../../preperation/SAVE_dataSet'
dataSetDir = join(relativeDir,nameDataset)
# 모델 저장 위치
SaveDir = '/restricted/projectnb/movelab/bcha/IMUforKnee/trainedModel/'

# tensorboard 위치
totalFold = 5
epochs = 1000
lreaningRate = 0.002
batch_size = 32
log_interval = 10
############################
# 시간 설정
time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [17]:
# CPU or GPU?
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print(DEVICE)

# 모델 만들기
class Mlp(nn.Module):
    def __init__(self):
        super(Mlp, self).__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Linear(4242,6000)
        self.layer2 = nn.Linear(6000,4000)
        self.layer3 = nn.Linear(4000,303)
        self.dropout1 = nn.Dropout(p=0.5)
        
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.layer1(x))
        x = self.dropout1(x)
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x


cpu


In [18]:
class Dataset(torch.utils.data.Dataset): 
  def __init__(self, dataSetDir, dataType, sess, numFold):
      self.dataType = dataType # angle or moBWHT
      self.sess = sess # train or test
      self.load_Data = np.load(join(dataSetDir,f"{str(numFold)}_fold_final_{sess}.npz"))

  def __len__(self):
      return len(self.load_Data[f'final_X_{self.sess}'])

  def __getitem__(self, idx):
    X = torch.squeeze(torch.FloatTensor(self.load_Data[f'final_X_{self.sess}'][idx]))
    Y = torch.squeeze(torch.FloatTensor(self.load_Data[f'final_Y_{self.dataType}_{self.sess}'][idx]))
    return X, Y

In [19]:
def nRMSE_Axis_Perbatch(pred, target, dataType, axis, numFold):
    dict_axis = {
    'x': 0,
    "y": 1,
    "z": 2,
    }
    axis = dict_axis[axis]
    nRMSE_perbatch = 0
    # 필요한 sclaer 불러오기
    load_scaler4Y = load(open(join(dataSetDir,f"{numFold}_fold_scaler4Y_{dataType}.pkl"), 'rb'))
    batchNum = len(target)
    for bat in range(batchNum): # batch 내를 순회
        pred_axis   = torch.transpose(torch.reshape(torch.squeeze(pred[bat]), [3,-1]), 0, 1)[:,axis]
        target_axis = torch.transpose(torch.reshape(torch.squeeze(target[bat]), [3,-1]), 0, 1)[:,axis]
        pred_axis = (pred_axis - torch.tensor(load_scaler4Y.min_[axis])) / torch.tensor(load_scaler4Y.scale_[axis])
        target_axis = (target_axis - torch.tensor(load_scaler4Y.min_[axis])) / torch.tensor(load_scaler4Y.scale_[axis])
        nRMSE = 100 * torch.sqrt(torch.mean(torch.square(pred_axis - target_axis))) / (torch.max(target_axis) - torch.min(target_axis))
        nRMSE_perbatch += nRMSE
    nRMSE_perbatch /= batchNum
    return nRMSE_perbatch
    
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

In [23]:
layout = {
    "ABCDE": {
        "loss": ["Multiline", ["loss(MAE)/training", "loss(MAE)/validation"]],
        "X_nRMSE": ["Multiline", [f'{dataType}_X_nRMSE/training', f"{dataType}_X_nRMSE/validation"]],
        "Y_nRMSE": ["Multiline", [f'{dataType}_Y_nRMSE/training', f"{dataType}_Y_nRMSE/validation"]],
        "Z_nRMSE": ["Multiline", [f'{dataType}_Z_nRMSE/training', f"{dataType}_Z_nRMSE/validation"]],
    },
}


for numFold  in range(totalFold):
    print(f'now fold: {numFold}')
    # 매 fold마다 새로운 모델
    my_model = Mlp()
    my_model.to(DEVICE)
    

    # loss function and optimizer define
    criterion = nn.L1Loss() # mean absolute error
    optimizer = torch.optim.NAdam(my_model.parameters(),lr=lreaningRate)

    angle_train = Dataset(dataSetDir, dataType, 'train',numFold)
    angle_test  = Dataset(dataSetDir, dataType, 'test', numFold)
    train_loader = DataLoader(angle_train, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(angle_test, batch_size=batch_size, shuffle=True, pin_memory=True)
    

    # 시각화를 위한 tensorboard 초기화
    writer = SummaryWriter(f'./logs/pytorch/{modelVersion}/{nameDataset}/{numFold}_fold')
    writer.add_custom_scalars(layout)
    x = torch.rand(1, 4242, device=DEVICE)
    writer.add_graph(my_model,x)
    # 학습시작
    for epoch in range(epochs):
        my_model.train()
        
        train_loss = 0
        train_x_nRMSE = 0
        train_y_nRMSE = 0
        train_z_nRMSE = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = my_model(data)
            loss = criterion(output, target)
            train_loss += loss.item() # 원본코드에는 수정됨
            train_x_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'x')
            train_y_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'y')
            train_z_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'z')
            loss.backward()
            optimizer.step()

            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

            
        train_loss /= len(train_loader) 
        train_x_nRMSE /= len(train_loader) 
        train_y_nRMSE /= len(train_loader) 
        train_z_nRMSE /= len(train_loader) 
        writer.add_scalar('loss(MAE)/training', train_loss, epoch)
        writer.add_scalar(f'{dataType}_X_nRMSE/training', train_x_nRMSE, epoch)
        writer.add_scalar(f'{dataType}_Y_nRMSE/training', train_y_nRMSE, epoch)
        writer.add_scalar(f'{dataType}_Z_nRMSE/training', train_z_nRMSE, epoch)


        my_model.eval()  # batch norm이나 dropout 등을 train mode 변환
        test_loss = 0
        test_x_nRMSE = 0
        test_y_nRMSE = 0
        test_z_nRMSE = 0

        with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
            for data, target in test_loader:
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = my_model(data)
                test_loss += criterion(output,target).item()
                test_x_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'x')
                test_y_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'y')
                test_z_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'z')                

            test_loss /= len(test_loader)
            writer.add_scalar('loss(MAE)/validation', test_loss, epoch)
            writer.add_scalar(f'{dataType}_X_nRMSE/validation', test_x_nRMSE, epoch)
            writer.add_scalar(f'{dataType}_Y_nRMSE/validation', test_y_nRMSE, epoch)
            writer.add_scalar(f'{dataType}_Z_nRMSE/validation', test_z_nRMSE, epoch)
        print(f'\nTrain set: Average loss: {train_loss:.4f}, X_nRMSE: {train_x_nRMSE}, Y_nRMSE: {train_y_nRMSE}, Z_nRMSE: {train_z_nRMSE}'
             +f'\nTest set: Average loss: {test_loss:.4f}, X_nRMSE: {test_x_nRMSE}, Y_nRMSE: {test_y_nRMSE}, Z_nRMSE: {test_z_nRMSE}')
        break
    writer.close()
    dir_save_torch = join(SaveDir,modelVersion,nameDataset)
    ensure_dir(dir_save_torch)
    model_scripted = torch.jit.script(my_model) # Export to TorchScript
    model_scripted.save(join(dir_save_torch,f'{dataType}_{numFold}_fold.pt')) # Save
    # 저장된 모델 불러올 때
    # 항상 불러온 모델 뒤에 model.eval() 붙일 것!
    # https://tutorials.pytorch.kr/beginner/saving_loading_models.html#export-load-model-in-torchscript-format
    # model = torch.jit.load('model_scripted.pt')
    # model.eval()



now fold: 0


In [ ]:

    # 시각화를 위한 tensorboard 초기화
    writer = SummaryWriter(f'./logs/pytorch/{modelVersion}/{nameDataset}/{numFold}_fold')
    writer.add_custom_scalars(layout)
    x = torch.rand(1, 4242, device=DEVICE)
    writer.add_graph(my_model,x)
    # 학습시작
    for epoch in range(epochs):
        my_model.train()
        
        train_loss = 0
        train_x_nRMSE = 0
        train_y_nRMSE = 0
        train_z_nRMSE = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = my_model(data)
            loss = criterion(output, target)
            train_loss += loss.item() # 원본코드에는 수정됨
            train_x_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'x')
            train_y_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'y')
            train_z_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'z')
            loss.backward()
            optimizer.step()

            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

            
        train_loss /= len(train_loader) 
        train_x_nRMSE /= len(train_loader) 
        train_y_nRMSE /= len(train_loader) 
        train_z_nRMSE /= len(train_loader) 
        writer.add_scalar('loss(MAE)/training', train_loss, epoch)
        writer.add_scalar(f'{dataType}_X_nRMSE/training', train_x_nRMSE, epoch)
        writer.add_scalar(f'{dataType}_Y_nRMSE/training', train_y_nRMSE, epoch)
        writer.add_scalar(f'{dataType}_Z_nRMSE/training', train_z_nRMSE, epoch)


        my_model.eval()  # batch norm이나 dropout 등을 train mode 변환
        test_loss = 0
        test_x_nRMSE = 0
        test_y_nRMSE = 0
        test_z_nRMSE = 0

        with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
            for data, target in test_loader:
                data, target = data.to(DEVICE), target.to(DEVICE)
                output = my_model(data)
                test_loss += criterion(output,target).item()
                test_x_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'x')
                test_y_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'y')
                test_z_nRMSE += nRMSE_Axis_Perbatch(output,target, dataType, 'z')                

            test_loss /= len(test_loader)
            writer.add_scalar('loss(MAE)/validation', test_loss, epoch)
            writer.add_scalar(f'{dataType}_X_nRMSE/validation', test_x_nRMSE, epoch)
            writer.add_scalar(f'{dataType}_Y_nRMSE/validation', test_y_nRMSE, epoch)
            writer.add_scalar(f'{dataType}_Z_nRMSE/validation', test_z_nRMSE, epoch)
        print(f'\nTrain set: Average loss: {train_loss:.4f}, X_nRMSE: {train_x_nRMSE}, Y_nRMSE: {train_y_nRMSE}, Z_nRMSE: {train_z_nRMSE}'
             +f'\nTest set: Average loss: {test_loss:.4f}, X_nRMSE: {test_x_nRMSE}, Y_nRMSE: {test_y_nRMSE}, Z_nRMSE: {test_z_nRMSE}')
        break
    writer.close()
    dir_save_torch = join(SaveDir,modelVersion,nameDataset)
    ensure_dir(dir_save_torch)
    model_scripted = torch.jit.script(my_model) # Export to TorchScript
    model_scripted.save(join(dir_save_torch,f'{dataType}_{numFold}_fold.pt')) # Save
    # 저장된 모델 불러올 때
    # 항상 불러온 모델 뒤에 model.eval() 붙일 것!
    # https://tutorials.pytorch.kr/beginner/saving_loading_models.html#export-load-model-in-torchscript-format
    # model = torch.jit.load('model_scripted.pt')
    # model.eval()



NameError: name 'A' is not defined

# 이번에 배운 것

In [ ]:
import torch
import torchvision
from sklearn.metrics import classification_report
from tqdm import tqdm


class Net(torch.nn.Module):
    """We create neural nets by subclassing the torch.nn.Module class in
    the newly defined class, we define 2 things:
    1) The network elements/layers; these are defined in the __init__ method
    2) The network behavior! What happens when we call our model on an input
    (here we call the input 'x')

    Our model is thus composed of 2 Conv and 2 Linear layers, each with a
    different size. When we call our model against an input example, we compute
    the output from each layer and in between we apply the ReLU function.
    """

    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Conv2d(in_channels=1, out_channels=2, kernel_size=5)
        self.layer2 = torch.nn.Conv2d(in_channels=2, out_channels=2, kernel_size=5)
        self.layer3 = torch.nn.Linear(800, 16)
        self.layer4 = torch.nn.Linear(16, 10)

    def forward(self, x):
        # pass through conv layers
        x = self.layer1(x)
        x = torch.nn.functional.relu(x)
        x = self.layer2(x)
        x = torch.nn.functional.relu(x)

        # pass through linear layers
        x = torch.flatten(x, start_dim=1)  # flatten the output of convolution
        x = self.layer3(x)
        x = torch.nn.functional.relu(x)
        x = self.layer4(x)
        return x


# we initialize our model as thus:
# Congrats! You just built a neural network with PyTorch :-)
my_model = Net()


# GPU-aware programming
"""
our PyTorch module loads automatically to CPU, and afterwards we can decide to
send it to GPU using .to() method. In fact tensor.to() method can send any
PyTorch tensor to any device, not just models.
"""
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")
print(DEVICE)
my_model.to(DEVICE)  # this sends the model to the appropriate device

# Get data
DATA_PATH = "data/FashionMNIST/"
# transforms
transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5,), (0.5,)),
    ]
)
train_ds = torchvision.datasets.FashionMNIST(
    root=DATA_PATH, train=True, transform=transform, download=True
)
test_ds = torchvision.datasets.FashionMNIST(
    root=DATA_PATH, train=False, transform=transform, download=True
)

"""Let's define our training data loader. We will adopt a batch size of 16.
Shuffling the data is also useful for training. 0 workers means that the data
will be loaded in the main process.
"""
train_loader = torch.utils.data.DataLoader(
    train_ds, batch_size=16, shuffle=True, num_workers=0
)
test_loader = torch.utils.data.DataLoader(
    test_ds, batch_size=16, shuffle=False, num_workers=0
)

# Let's create a translation from the class numbers to Human-Readable
# (and meaningful) text labels

LABEL_DICT = {
    0: "T-shirt/top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle boot",
}

"""There are many Optimizers in the PyTorch Library. We select the ADAM
optimizer as one of the most recognizable and efficient optimizers in the
Deep Learning field. We feed the optimizer object, the parameters which it
will optimize!
"""
optimizer = torch.optim.Adam(my_model.parameters())

"""There are many Loss functions in the PyTorch Library. We pick one that is
suitable for the problem we are working on (Image Classification). It is called
Cross Entropy Loss.
"""
criterion = torch.nn.CrossEntropyLoss()

"""We leverage the classification_report function in Sci-Kit Learn! More info
here: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
SK Learn has many other metrics, you are welcome to check them out and use as
needed for your problem.
"""


def compute_metrics(labels, preds, mode="train"):
    """print out standard metrics for classification."""
    # we use the values in the dictionary we defined earlier (see above)
    # to define a list of class names
    # this is helpful in making the report more meaningful!
    class_names = list(LABEL_DICT.values())
    labels = torch.cat(labels)  # concatenate list into tensor
    preds = torch.cat(preds)
    metrics = classification_report(
        y_true=labels,
        y_pred=preds,
        target_names=class_names,
        output_dict=True,
    )
    print(f"\n-------- begin report: {mode} ----------------------------\n")
    print(f"prediction sample: {preds[0:10]}, label sample: {labels[0:10]}")
    for key, value in metrics.items():
        print(key, value)
    print(f"\n-------- end report: {mode} -------------------------------\n")
    return metrics


def train_step(my_model, images, labels, optimizer, criterion):
    # zero out the gradient parameters
    optimizer.zero_grad()

    # forward + backward + optimize

    # forward step: compute model output
    prediction = my_model(images)

    # backward step: compute loss
    loss = criterion(prediction, labels)
    loss.backward()  # calculate mini-batch grads

    # optimizer step: update the model parameters
    optimizer.step()  # update weights afters mini-batch
    return prediction, loss


NUM_EPOCHS = 5  # how many times will we go over our data?

"""Some Notes:
1) tqdm is a helpful tool that provides nice "progress bar" graphic in the
output console :-) our work now looks professional
2) Models have 2 modes, training and evaluation. Since we're going to train
our model, we change the mode to train
"""
my_model.train()  # change model from eval mode to train mode

for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0  # reset epoch loss
    all_preds = []  # list of all predictions in the epoch
    all_labels = []  # list of all labels in the epoch
    num_batches = len(train_loader)
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}"):
        # get data
        images, labels = batch
        images = images.to(device=DEVICE, dtype=torch.float)
        labels = labels.to(device=DEVICE, dtype=torch.long)
        # we defined "device" when we defined the model, see above

        prediction, loss = train_step(my_model, images, labels, optimizer, criterion)
        epoch_loss += loss  # compile batch loss

        # metrics
        """Get the classification using the argmax function and send the tensor
        to CPU using .cpu() (or do nothing if they're already in CPU). This is
        important, our compute_metrics function expects CPU tensors
        """
        preds = torch.argmax(prediction, dim=1).cpu()
        labels = labels.cpu()
        all_preds.append(preds)
        all_labels.append(labels)
        # batch ends

    # epoch ends
    epoch_loss /= num_batches
    print("==================== train ==============")
    print(f"\n Train Loss: {epoch_loss} \n")

    # Compute metrics for classification
    metrics = compute_metrics(all_labels, all_preds)
print("finished training!")
# Congrats! you just trained your first neural network in PyTorch


# evaluation
my_model.eval()
with torch.no_grad():
    val_loss = 0.0  # reset epoch loss
    all_preds = []  # list of all predictions in the epoch
    all_labels = []  # list of all labels in the epoch
    num_batches = len(test_loader)
    for batch in tqdm(test_loader, desc=f"validation progress"):
        # get data
        images, labels = batch
        images = images.to(device=DEVICE, dtype=torch.float)
        labels = labels.to(device=DEVICE, dtype=torch.long)
        # we defined "device" when we defined the model, see above

        prediction = my_model(images)
        loss = criterion(prediction, labels)
        val_loss += loss  # compile batch loss

        # metrics
        preds = torch.argmax(prediction, dim=1).cpu()
        labels = labels.cpu()
        all_preds.append(preds)
        all_labels.append(labels)
        # batch ends

print("==================== evaluation ==============")
print("\n Validation Loss: %f \n" % val_loss)

metrics = compute_metrics(all_labels, all_preds, mode="val")